## Stage 1: Install dependencies and setting up GPU environment

In [1]:
!pip install numpy==1.16.1

     |████████████████████████████████| 17.3MB 203kB/s 
ERROR: umap-learn 0.4.6 has requirement numpy>=1.17, but you'll have numpy 1.16.1 which is incompatible.
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: numpy 1.18.5
    Uninstalling numpy-1.18.5:
      Successfully uninstalled numpy-1.18.5


## Stage 2: Importing project dependencies

In [2]:
import numpy as np
import tensorflow as tf
from keras.layers import Dropout
from tensorflow.keras.datasets import imdb

In [3]:
tf.__version__

'2.3.0'

## Stage 3: Dataset preprocessing

### Setting up dataset parameters

In [78]:
number_of_words = 20000
max_len = 100

### Loading the IMDB dataset

In [79]:
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=number_of_words)

### Padding all sequences to be the same length 

In [80]:
X_train = tf.keras.preprocessing.sequence.pad_sequences(X_train, maxlen=max_len)

In [81]:
X_test = tf.keras.preprocessing.sequence.pad_sequences(X_test, maxlen=max_len)

### Setting up Embedding Layer parameters

In [82]:
vocab_size = number_of_words
embed_size = 256
input_shape_emb = (X_train.shape[1],)

## Step 4: Building a Recurrent Neural Network

In [83]:
def add_embeding_layer(model,vocab_size, embed_size, input_shape):
  model.add(tf.keras.layers.Embedding(input_dim = vocab_size, output_dim = embed_size, input_shape=input_shape))
  return model

def add_lstm_layer(model, units=256, activation='tanh', drop_out_rate=0.2, return_sequences=True):
  model.add(tf.keras.layers.LSTM(units=units, activation=activation, return_sequences = return_sequences))
  model.add(Dropout(drop_out_rate))
  return model

def add_output_layer(model, units=1, activation='sigmoid'):
  model.add(tf.keras.layers.Dense(units=units, activation=activation))
  return model

In [84]:
model = tf.keras.Sequential()
model = add_embeding_layer(model,vocab_size, embed_size, input_shape)
model = add_lstm_layer(model, units=256, activation='tanh', drop_out_rate=0.4, return_sequences=True)
model = add_lstm_layer(model, units=128, activation='tanh', drop_out_rate=0.3, return_sequences=True)
model = add_lstm_layer(model, units=64, activation='tanh', drop_out_rate=0.2, return_sequences=True)
model = add_lstm_layer(model, units=32, activation='tanh', drop_out_rate=0.1, return_sequences=False)
model = add_output_layer(model)

### Compiling the model

In [85]:
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])

In [86]:
model.summary()

Model: "sequential_16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_13 (Embedding)     (None, 100, 256)          5120000   
_________________________________________________________________
lstm_25 (LSTM)               (None, 100, 256)          525312    
_________________________________________________________________
dropout_18 (Dropout)         (None, 100, 256)          0         
_________________________________________________________________
lstm_26 (LSTM)               (None, 100, 128)          197120    
_________________________________________________________________
dropout_19 (Dropout)         (None, 100, 128)          0         
_________________________________________________________________
lstm_27 (LSTM)               (None, 100, 64)           49408     
_________________________________________________________________
dropout_20 (Dropout)         (None, 100, 64)         

### Training the model

In [87]:
model.fit(X_train, y_train, epochs=7, batch_size=64)

Epoch 1/7
391/391 [==============================] - 24s 62ms/step - loss: 0.4593 - accuracy: 0.7885
Epoch 2/7
391/391 [==============================] - 24s 62ms/step - loss: 0.2931 - accuracy: 0.8830
Epoch 3/7
391/391 [==============================] - 24s 62ms/step - loss: 0.2306 - accuracy: 0.9112
Epoch 4/7
391/391 [==============================] - 24s 62ms/step - loss: 0.1801 - accuracy: 0.9341
Epoch 5/7
391/391 [==============================] - 24s 62ms/step - loss: 0.1380 - accuracy: 0.9516
Epoch 6/7
391/391 [==============================] - 24s 62ms/step - loss: 0.1018 - accuracy: 0.9643
Epoch 7/7
391/391 [==============================] - 24s 62ms/step - loss: 0.0723 - accuracy: 0.9767


### Evaluating the model

In [88]:
test_loss, test_acurracy = model.evaluate(X_test, y_test)

782/782 [==============================] - 11s 14ms/step - loss: 0.7851 - accuracy: 0.8201


In [89]:
print("Test accuracy: {}".format(test_acurracy))

Test accuracy: 0.8201199769973755
